# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [11]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [12]:
stocks = pd.read_csv('sp_500_stocks.csv')
from tokenz import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [17]:
# symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(api_url).json()
# data

symbol='aapl'
url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
outer_data = requests.get(url).json()
outer_data

[{'beta': 1.0391476142440639,
  'totalCash': 67150000000,
  'currentDebt': 123822000000,
  'revenue': 381623000000,
  'grossProfit': 173966000000,
  'totalRevenue': 381623000000,
  'EBITDA': 129629000000,
  'revenuePerShare': 24.89,
  'revenuePerEmployee': 2596074.83,
  'debtToEquity': 4.54768579669515,
  'profitMargin': 0.2630580441954494,
  'enterpriseValue': 2893872794960,
  'enterpriseValueToRevenue': 7.58,
  'priceToSales': 7.48,
  'priceToBook': 38.5,
  'forwardPERatio': 28.24659325127216,
  'pegRatio': 3.0530400663553263,
  'peHigh': 11.867282676107855,
  'peLow': 7.976472620192518,
  'week52highDate': '2023-12-14',
  'week52lowDate': '2024-04-19',
  'putCallRatio': None,
  'companyName': 'Apple Inc',
  'marketcap': 2856432794960,
  'week52high': 198.57,
  'week52low': 163.63,
  'week52highSplitAdjustOnly': 199.62,
  'week52highDateSplitAdjustOnly': '2023-12-14',
  'week52lowSplitAdjustOnly': 164.08,
  'week52lowDateSplitAdjustOnly': '2024-04-19',
  'week52change': 0.09411002536

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [20]:
data = outer_data[0]
# data
# data['avgTotalVolume']
data['year1ChangePercent']

0.09094018335385036

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [37]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker'][:5]:
    # print(symbol)
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(batch_api_call_url).json()
    dataOne = outer_data[0]
    dataTwo = outer_data[1]
    # print(dataOne['latestPrice'])
    # print(dataTwo['year1ChangePercent'])
    new_row = pd.Series([symbol, dataOne['latestPrice'], dataTwo['year1ChangePercent'],'N/A'], index = my_columns)
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_14880\2475578383.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)


In [38]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,151.28,0.167409,N/A
1,AAL,15.19,0.064907,N/A
2,AAP,75.64,-0.377378,N/A
3,AAPL,187.43,0.090940,N/A
4,ABBV,161.59,0.140514,N/A


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [25]:
# # Function sourced from 
# # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]   
        
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [33]:
for symbol in stocks['Ticker'][:1]:
    # print(symbol_strings)
    # batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(batch_api_call_url).json()
    dataOne = outer_data[0]
    dataTwo = outer_data[1]
    print(dataOne['latestPrice'])
    print(dataTwo['year1ChangePercent'])

151.28
0.1674089383894719


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [23]:
# final_dataframe = pd.DataFrame(columns = my_columns)

# for symbol_string in symbol_strings:
# #     print(symbol_strings)
#     # batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#     batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
#     outer_data = requests.get(batch_api_call_url).json()
#     dataOne = outer_data[0]
#     dataTwo = outer_data[1]

#     for symbol in symbol_string.split(','):
#         # new_row = pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['stats']['year1ChangePercent'],'N/A'], index = my_columns)
#         new_row = pd.Series([symbol, dataOne['latestPrice'], dataTwo['year1ChangePercent'],'N/A'], index = my_columns)
#         final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
        
    
# final_dataframe

TypeError: list indices must be integers or slices, not str

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [39]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:3]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,151.28,0.167409,N/A
1,ABBV,161.59,0.140514,N/A
2,AAPL,187.43,0.090940,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [40]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

1000


In [41]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,151.28,0.167409,2
1,ABBV,161.59,0.140514,2
2,AAPL,187.43,0.090940,1


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [70]:
# hqm_columns = [
#                 'Ticker', 
#                 'Price', 
#                 'Number of Shares to Buy', 
#                 'One-Year Price Return', 
#                 'One-Year Return Percentile',
#                 'Six-Month Price Return',
#                 'Six-Month Return Percentile',
#                 'Three-Month Price Return',
#                 'Three-Month Return Percentile',
#                 'One-Month Price Return',
#                 'One-Month Return Percentile',
#                 'HQM Score'
#                 ]

# hqm_dataframe = pd.DataFrame(columns = hqm_columns)

# for symbol_string in symbol_strings:
# #     print(symbol_strings)
#     batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#     data = requests.get(batch_api_call_url).json()
#     for symbol in symbol_string.split(','):
#         hqm_dataframe = hqm_dataframe.append(
#                                         pd.Series([symbol, 
#                                                    data[symbol]['quote']['latestPrice'],
#                                                    'N/A',
#                                                    data[symbol]['stats']['year1ChangePercent'],
#                                                    'N/A',
#                                                    data[symbol]['stats']['month6ChangePercent'],
#                                                    'N/A',
#                                                    data[symbol]['stats']['month3ChangePercent'],
#                                                    'N/A',
#                                                    data[symbol]['stats']['month1ChangePercent'],
#                                                    'N/A',
#                                                    'N/A'
#                                                    ], 
#                                                   index = hqm_columns), 
#                                         ignore_index = True)
        
# hqm_dataframe.columns

hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol in stocks['Ticker'][:5]:
    # print(symbol)
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(batch_api_call_url).json()
    dataOne = outer_data[0]
    dataTwo = outer_data[1]
    new_row = pd.Series([symbol, dataOne['latestPrice'], 'N/A', dataTwo['year1ChangePercent'], 'N/A', dataTwo['month6ChangePercent'],'N/A',dataTwo['month3ChangePercent'], 'N/A',dataTwo['month1ChangePercent'],'N/A','N/A'], index = hqm_columns)
    hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_14880\1429134098.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame([new_row])], ignore_index=True)


In [71]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.28,N/A,0.167409,N/A,0.379586,N/A,0.140568,N/A,0.049954,N/A,N/A
1,AAL,15.19,N/A,0.064907,N/A,0.267402,N/A,0.021903,N/A,0.135361,N/A,N/A
2,AAP,75.64,N/A,-0.377378,N/A,0.355975,N/A,0.172470,N/A,0.068085,N/A,N/A
3,AAPL,187.43,N/A,0.090940,N/A,0.013325,N/A,0.009433,N/A,0.057975,N/A,N/A
4,ABBV,161.59,N/A,0.140514,N/A,0.185383,N/A,-0.060614,N/A,-0.006162,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [72]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0    1.0
1    0.4
2    0.2
3    0.6
4    0.8
Name: One-Year Return Percentile, dtype: object
0    1.0
1    0.6
2    0.8
3    0.2
4    0.4
Name: Six-Month Return Percentile, dtype: object
0    0.8
1    0.6
2    1.0
3    0.4
4    0.2
Name: Three-Month Return Percentile, dtype: object
0    0.4
1    1.0
2    0.8
3    0.6
4    0.2
Name: One-Month Return Percentile, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.28,N/A,0.167409,1.0,0.379586,1.0,0.140568,0.8,0.049954,0.4,N/A
1,AAL,15.19,N/A,0.064907,0.4,0.267402,0.6,0.021903,0.6,0.135361,1.0,N/A
2,AAP,75.64,N/A,-0.377378,0.2,0.355975,0.8,0.172470,1.0,0.068085,0.8,N/A
3,AAPL,187.43,N/A,0.090940,0.6,0.013325,0.2,0.009433,0.4,0.057975,0.6,N/A
4,ABBV,161.59,N/A,0.140514,0.8,0.185383,0.4,-0.060614,0.2,-0.006162,0.2,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [73]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [74]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:2]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [75]:
portfolio_input()

In [76]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.28,3,0.167409,1.0,0.379586,1.0,0.140568,0.8,0.049954,0.4,0.8
1,AAL,15.19,N/A,0.064907,0.4,0.267402,0.6,0.021903,0.6,0.135361,1.0,0.65


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()